In [10]:
!pip install -U sagemaker

     |████████████████████████████████| 435 kB 24.4 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.52.0-py2.py3-none-any.whl size=613289 sha256=aceaf2a7515858effa1ffd323c4b43454d6502bd962c51c688a8dfc4d23dbf37
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1b/0b/05/f42f221810f419089bb19bcde0555c5e36f975f30423fadd99
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.45.0
    Uninstalling sagemaker-2.45.0:
      Successfully uninstalled sagemaker-2.45.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
import pandas as pd
import sagemaker
tf.__version__

#Reading in data in folder
df = pd.read_csv('Iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
#Splitting data in 80-20 split to use testing data for model inference later
train = df.iloc[:120,:]
test = df.iloc[121:,:]
train = train.drop('Id', axis=1)
test = test.drop('Id', axis=1)

#Train and test csv
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [5]:
#Create a sagemaker session to be able to upload data to s3
import boto3
sagemaker_session = sagemaker.Session()

#Uploading data to S3 bucket titled "tf-iris-data"
prefix = "tf-iris-data"
training_input_path = sagemaker_session.upload_data('train.csv', key_prefix=prefix + '/training')
training_input_path

's3://sagemaker-us-east-1-277564125686/tf-iris-data/training/train.csv'

In [8]:
#Need tf version and role for the Tensorflow Estimator
tf_version = tf.__version__
print(tf_version)

#Sagemaker role, make sure you've allowed access to any S3 bucket
role = sagemaker.get_execution_role()
role

2.3.2


'arn:aws:iam::277564125686:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'

In [19]:
# Use a tensorflow estimator from sagemaker to train model
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.c5.18xlarge',
                          framework_version=tf_version, 
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 30
                          }
                         )

#Training
tf_estimator.fit({'train': training_input_path})

2021-08-06 10:50:51 Starting - Starting the training job...
2021-08-06 10:51:14 Starting - Launching requested ML instancesProfilerReport-1628247050: InProgress
......
2021-08-06 10:52:15 Starting - Preparing the instances for training......
2021-08-06 10:53:17 Downloading - Downloading input data...
2021-08-06 10:53:44 Training - Downloading the training image..2021-08-06 10:54:00.839473: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-08-06 10:54:00.843407: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-08-06 10:54:01.007739: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-08-06 10:54:03,728 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-08-06 10:54:03,738 sagemaker-training-

In [22]:
#Creating endpoint and deploying model
import time
tf_endpoint_name = 'tf-iris-model'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
tf_predictor = tf_estimator.deploy(initial_instance_count=1,instance_type='ml.m5.4xlarge',
                                   endpoint_name=tf_endpoint_name)
print("Success!")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!Success!


In [23]:
#Sample test
predictions = tf_predictor.predict([5.1,3.5,1.4,0.2])['predictions'] #returns a list of lists

#Inference function to print model result
def inferenceFunction(preds):
    preds = max(preds)
    maxValue = max(preds)
    maxIndex = preds.index(maxValue)
    if maxIndex == 0:
        print("Iris-setosa")
    elif maxIndex == 1:
        print("Iris-versicolor")
    else:
        print("Iris-virginica")

results = inferenceFunction(predictions)
print(results)

Iris-versicolor
None


In [26]:
predictions = tf_predictor.predict([5.1,3.5,1.4,0.2])#returns a list of lists
print(predictions)
print(dir(tf_predictor))

{'predictions': [[0.0353562422, 0.90780127, 0.0568424836]]}
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_classify_or_regress', '_context', '_create_request_args', '_delete_endpoint_config', '_endpoint_config_name', '_get_endpoint_config_name', '_get_model_monitor_class', '_get_model_names', '_handle_response', '_model_attributes', '_model_names', 'accept', 'classify', 'content_type', 'delete_endpoint', 'delete_model', 'deserializer', 'disable_data_capture', 'enable_data_capture', 'endpoint', 'endpoint_context', 'endpoint_name', 'list_monitors', 'predict', 'regress', 'sagemaker_session', 'serializer', 'update_data_capture_config', 'update_endpoint']
